In [ ]:
from pathlib import Path


# classes_path = Path('../input/train_simplified/')
cls_path = Path('../input/train_simplified/')
# iter_cls_path = cls_path.iterdir()

class_dict = {x.stem.replace(" ", "_"):i for i, x in enumerate(cls_path.iterdir())}
labels = {x.stem.replace(" ", "_") for i, x in enumerate(cls_path.iterdir())}

n_labels = len(labels)
print("Number of labels: {}".format(n_labels))

In [ ]:
import pdb

max_img_per_class = 10
wshape = 64
hshape = 64
# to image from stroke
def drawing_to_np(drawing, shape=(64, 64)):
    drawing = ast.literal_eval(drawing)
    fig, ax = plt.subplots()
    for x,y in drawing:
        ax.plot(x, y, marker='.')
        ax.axis('off')
    
    fig.canvas.draw()
    # Convert images to numpy arrat
    np_drawing = np.array(fig.canvas.renderer._renderer)
    plt.close(fig)
#     pdb.set_trace()
    img = cv2.resize(np_drawing, shape)
    img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
#     img_expanded = img_gray[:, :, np.newaxis]
#     img_expanded = img_gray[np.newaxis, :, :]
#     pdb.set_trace()
    return img_gray
#     return img_expanded
#     return img

def path_to_img(path):
    train = pd.read_csv(path, usecols=['word', 'drawing'], nrows=max_img_per_class)
    train['word'] = train.word.str.replace(" ", "_")
    train['drawing_np'] = train['drawing'].map(drawing_to_np)
    label = train.word.map(class_dict)
    return  train['drawing_np'], label



In [ ]:
# next(cls_path.iterdir()).stat().st_size

def save_image(image_numpy, image_path):
    image_pil = Image.fromarray(image_numpy)
    image_pil.save(image_path)

def csv2img(fn, save_path=Path('.')):
    n_row = 100
    train = pd.read_csv(fn, usecols=['word', 'drawing'], nrows=n_row)
    train['word'] = train.word.str.replace(" ", "_")
    train['drawing_np'] = train['drawing'].apply(drawing_to_np, True, shape=(128, 128))
#     label = train.word.map(class_dict)
#     return  train['drawing_np'], label
    for i, np_drawing in enumerate(train['drawing_np']):
        save_image(np_drawing, save_path/f'car{i}.png')
        
    

In [ ]:
# keras generator 

fileList = glob("../input/train_simplified/*.csv")     
def imageLoader(files, batch_size):

    L = len(files*max_img_per_class)

    while True:

        batch_start = 0
        batch_end = batch_size

        while batch_start < L:
            limit = min(batch_end, L)
        
            Xs, Ys = map(path_to_img, files[batch_start:limit])
            
            for X, Y in zip(Xs, Ys):
                yield (X,Y)    
                batch_start += batch_size   
                batch_end += batch_size
           